In [ ]:
import warnings
warnings.filterwarnings('ignore')

# data imports
import pandas as pd
import numpy as np
from plotnine import *

# modeling imports
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, LogisticRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler, OneHotEncoder #Z-score variables
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error, accuracy_score #model evaluation
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut

# pipeline imports
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer

%matplotlib inline


# Review
## Time and Space Complexity (Vibes)

- Think about how a linear regression model makes predictions: When we train a Linear Regression model, what *information* do we need to store in order to make a **prediction** for a future person/data point?
- If we add an additional data point (row) when **training**, how would that impact what we'd need to store to make a prediction? Would prediction be faster (theoretically) or slower?
- If we add an additional feature (column) when **training**, how would that impact what we'd need to store to make a prediction? Would prediction be faster (theoretically) or slower?



## Link Functions

### Generally
<img src="https://drive.google.com/uc?export=view&id=1X_I73ds6NXjqZ7Mt62-YZztwm1O5dID5" alt="Q" width = "200"/>

In our Linear Regression lectures, we talked about adding non-linearity through **Feature Engineering**, but that's not the only way! We can also use **link functions** to add non-linearity.

Link functions are just algebra we do to the linear prediction ($\mathbf{X}\beta$) in order to get the predicted value we *actually* want (e.g a probability). 

$$\underbrace{y = \mathbf{X}\beta}_\text{Linear Model}$$
$$\underbrace{y = g^{-1}(\mathbf{X}\beta)}_\text{Generalized Linear Model}$$

Oddly, we often specify our link function using it's *inverse*, hence the $g^{-1}()$ instead of $g()$. $g^{-1}()$ takes the linear prediction and transforms it into our desired predicted value. $g()$ takes our desired predicted value and transforms it back into our linear prediction.



### Logistic Regression
In logistic regression, our goal is to predict a *probability* that a data point is in group `1`. We talked about using:

- Linear Probability Models $g^{-1}: y = x$
- Odds Models $g^{-1}: y = e^x$
- Logistic Regression: $g^{-1}: y = \frac{e^x}{1 + e^x}$

Logistic Regression using the link function $g(x) = log{\frac{x}{1-x}}$ and inverse link $g^{-1}: y = \frac{e^x}{1 + e^x}$ gave us a great sigmoid shape that takes linear predictions ($y = \mathbf{X}\beta$) and turns them into predicted probabilities ($p = \frac{e^{\mathbf{X}\beta}}{1 + e^{\mathbf{X}\beta}}$).


## Linear Probability is Fine SOMETIMES $^*$

<img src="https://drive.google.com/uc?export=view&id=1uQtIsF3Sx_ZHxqAg_bcJiktQABXfB1ML" alt="Q" width = "100"/>

When I got my dog, Nova, she was 8 weeks old, and the vet said that we shouldn't walk her too much at first, but we could walk her 5 minutes per month old she was (e.g. at 1 month, she can walk 5 minutes at a time. at 2 months she could walk 10...etc.). 

The vet gave us a *linear* model to predict how much our dog could walk. And it worked for about 8 months...but then what?

<img src="https://drive.google.com/uc?export=view&id=1yXaVVHhFewTOQ_gzFVLOamSCB2rdb2cm" alt="Q" width = "250"/>

<img src="https://drive.google.com/uc?export=view&id=1-TjUHcckruKEpDYMPeEDtArdqd5umXrq" alt="Q" width = "250"/>

<img src="https://drive.google.com/uc?export=view&id=1njdIwVpJOu8O7xh9PzR-rA4JI4pWt0vp" alt="Q" width = "250"/>


## Maximum Likelihood Estimation

Just like with Linear Regression, we can use **Maximum Likelihood Estimation** to choose the parameters (intercept and coefficients) of the model. But we have a different likelihood.

In a linear regression, we assumed that our *errors* are *normally* distributed around the regression line. For logistic regression, we assume that our *errors* are *Bernoulli* distributed. The Bernoulli distribution is a discrete distribution (since our outcome is *discrete*, a.k.a categorical) that tells you the proability of being `0` or `1`.

### Bernoulli Likelihood
The formula for a Bernoulli distribution for a single data point $x$ is:

$$ f(y;p(x)) = p(x)^{y} * (1-p(x))^{1-y}$$

where $y$ is the group the data point belongs to (either `0` or `1`), and $p(x)$ is the predicted probability of that data point being a `1`.

For example, let's say we're looking at the probability that it's sunny tomorrow. The predicted probability, according to the weather channel is $p(x) = 0.8$. The likelihood of it being sunny ($k = 1$) is:

$$ f(1;0.8) = 0.8^1 * (1-0.8)^{1-1} = 0.8$$

The likelihood of it not being sunny ($k = 0$) is: 
$$ f(0;0.8) = 0.8^0 * (1-0.8)^{1-0} = 0.2$$

### Likelihood Function
But we don't just have a SINGLE data point when fitting a logistic regression, we have MANY. So, we multiply the likelihood of each data point together to get the likelihood of the dataset: 

$$\prod_{i = 1}^n p(x_i)^{y_i} * (1-p(x_i))^{1-y_i}$$

We want to choose parameters (e.g. $\beta_0$, or $\beta_1$) that *maximize* this likelihood function. And how to we maximize it? We take it's (partial) derivatives and set them equal to zero!

However, it turns out that its much easier to work with the *log* of this likelihood function, so we're often working with the *log* likelihood and taking it's derivatives (this will still find the optimal parameters for the model as the values that maximize the *log* likelihood will also maximize the likelihood):

$$\sum_{i = 1}^n y_i * log(p(x_i)) + (1-y_i) * log(1-p(x_i))$$

### Loss Function
Now it turns out, this log-loss is a really great **loss function** for logistic regression. Loss functions are metrics that

1. measure the performance of your model, and
2. have lower scores indicate better performing models

Log-Loss (also called **Binary Cross Entropy**) does just that! Thus we often use it as a loss function for Logistic Regression. 

<img src="https://drive.google.com/uc?export=view&id=1jZBGKZCaDfP8g3mFpt0FArPc1r7H60Bs" alt="Q" width = "400"/>


# `sklearn`

Let's build a Logistic Regression using `sklearn`. We're still going to follow the same `sklearn` workflow that we've been using for Linear Regression:

1. Separate your data into X (predictors) and y (outcome), and maybe do some model validation set up.
2. Create an Empty Model.
3. call `.fit()` using your training data
4. call `.predict()` on ANY X data to get the model prediction for that data.
5. Assess the model

## Linear Regression -> Logistic Regression
And there's not that much to change, let's change this Linear Regression example to a Logistic Regression.

In [ ]:
pop = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/PopDivas_data.csv")
# pop.head()

# drop missing values
# pop.isnull().sum()
pop.dropna(inplace = True)
pop.reset_index(inplace = True)

# set up X and y
predictors = ["energy", "loudness",
              "speechiness", "acousticness",
              "instrumentalness", "liveness",
              "valence", "duration_ms",
              "artist_name"]
contin = ["energy", "loudness",
          "speechiness", "acousticness",
          "instrumentalness", "liveness",
          "valence", "duration_ms"]

X = pop[predictors]
y = pop["danceability"]

# validation

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

# z score
preprocess = make_column_transformer((StandardScaler(), contin),
                            (OneHotEncoder(), ["artist_name"]),
                            remainder = "passthrough")

lr = LinearRegression()

pipe = Pipeline([("pre", preprocess),
                ("linearregression", lr)])

# fit
pipe.fit(X_train,y_train)

# predict
y_pred_train = pipe.predict(X_train)
y_pred_test = pipe.predict(X_test)

# assess
print("Train MSE : ", mean_squared_error(y_train,y_pred_train))
print("Train MAE : ", mean_absolute_error(y_train,y_pred_train))
print("Train MAPE: ", mean_squared_error(y_train,y_pred_train))
print("Train R2  : ", r2_score(y_train,y_pred_train))

print("Test MSE  : ", mean_squared_error(y_test,y_pred_test))
print("Test MAE  : ", mean_absolute_error(y_test,y_pred_test))
print("Test MAPE : ", mean_squared_error(y_test,y_pred_test))

## Another Logistic Regression Example

Let's do another example from scratch using this [Breast Cancer](https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/BreastCancer.csv) data. 

In [ ]:
# read in data
bc = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/BreastCancer.csv")

bc.head()

# set X and y
predictors = bc.columns[bc.columns.str.endswith("mean")]

X = bc[predictors]
y = bc["diagnosis"]

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

# Create Empty Model


# fit


# predict

# assess


Quick note about Logistic Regression Coefficients:

1. Logistic Regression coefficients are by default in terms of *log odds* meaning that they tell you how much the *predicted log odds* of being in group 1 will change when the *predictor* increases by 1-unit.
2. In `sklearn`, Logistic Regression can actually handle outcomes with *more than 2* categories. For example we could predict what someone's major is using Logistic Regression. When we do that, `sklearn` actually builds *many* logistic regression models by default: one model per category (see [here](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) for more rigorous detail.) That means there are *multiple* sets of coefficients, one for each model. Because of this, when there is only 2 categories (e.g. a *binary* outcome), we use `[0]` to indicate that we want to grab only one set of coeficients, because there is only one set in the *binary* case.

In [ ]:
coefficients = pd.DataFrame({"Coef": pipe.named_steps['logreg'].coef_[0], # grab array of coefficients
                            "Name": predictors})
intercept = pd.DataFrame({"Coef": pipe.named_steps['logreg'].intercept_[0], # grab intercept
                                   "Name": "intercept"},
                                   index = [coefficients.shape[0]]) # since this is only 1 row, assign row a row index

coefficents_all = pd.concat([coefficients,intercept])
coefficents_all

# In Your Groups

## Practice Code
Now practice building Logistic Regression models in your groups. Using [this](https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/purchase.csv) dataset, build a Logistic Regression model that predicts whether or not customers signed up for a rewards program based on their age, income, and whether they had made a previous purchase. Make sure to z-score your continuous variables, and use an 80/20 Train-Test-Split.

For reasons you'll see in the next section **DO NOT Z-SCORE**.

In [ ]:
# read in data
df = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/purchase.csv")

df.head()

# set X and y
predictors = ["age", "income_in_k", "previous_purchase"]

X = df[predictors]
y = df["signed_up"]

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 392)

# Create Empty Model


# fit


# predict


# assess


Now, grab the coefficients for the model:

In [ ]:
# coefs
coefficients = pd.DataFrame({"Coef": pipe.named_steps['logreg'].coef_[0], # grab array of coefficients
                            "Name": predictors})
intercept = pd.DataFrame({"Coef": pipe.named_steps['logreg'].intercept_[0], # grab intercept
                                   "Name": "intercept"},
                                   index = [coefficients.shape[0]]) # since this is only 1 row, assign row a row index

coefficents_all = pd.concat([coefficients,intercept])
coefficents_all

## The Problem with Logistic Regression Coefficients
When you're presenting your Logistic Regression Models to non-data people, you might want to be able to tell them which variables have the biggest impact on the predicted value. Typically, we might use coefficients for this because they give us a single number that summarizes the relationship between our *predictors* and our *predicted value*. 

However, log odds are difficult to understand intuitively, especially if you're not a data person. Thus, we might want a different way to present our results. Luckily, if we *exponentiate* our **log odds** coefficients, we get **odds** coefficients. These are easier to understand, as most people understand intuitively what **odds** are. 

Remember, for **odds** the important threshold value is $1$. So any **odds** coefficient $>1$ has a direct/positive relationship with the outcome and anything with an **odds** coefficient $< 1$ has an inverse/negative relationship with the outcome. 

You can also use the **odds** coefs to give people an intuitive understanding of the relationship. If the odds coef is $2$ then increasing the predictor by 1-unit causes your predicted odds to *double*. Similarly, if the odds coef is $0.5$ then increasing the predictor by 1-unit causes your predicted odds to *halve*. If the odds coef is $1.25$ then increasing the predictor by 1-unit causes your predicted odds to increase by $25\%$.


### Question
Using [this app](https://shinylive.io/py/app/#h=0&code=NobwRAdghgtgpmAXGKAHVA6VBPMAaMAYwHsIAXOcpMAMwCdiYACAZwAsBLCbJjmVYnTJMAgujxM6lACZw6EgK4cJUqITIcAbnAA6EPgKFMIC-jygtjqPQcHDUUCNItMXqaXr2yaTVFOkc6gAUUADmcBJcJPASqAp0hGwWEUwANlAARnCpTAC8TDpgAApyLKRMAIyFAJSIekwNTCRwPvkQmFB0dFDYQcAAtADsGADMABwAbGNjI3gADBhzgwAsc3PLAKwAnBILcyPLI1vLKxIVGIMTAExbgyOXALrV9Y1+cAHqgpZtHV09fRUJGEUlFGCk4gkkiw4E8Xg1UsQ8lYMCxTEFmj4AFS+fyBMhfZ4QRqSOBkeIQILuDAAESgZCgADFuvAgiBCoVEExgOksqkHng4cShcLhYUADLEUJMADy0mkLEKTE53OI-MFIo1QsKsvlHK57QwcAAHqgggjqmqiZrrcTCkUGBlMhxUhwyNg9cADcbTeaAPRBCoAai9JrNxGqFoAvgKrTbrVxZEakcA5k9CbboOgAPpKJFKLDArM0VJKaRBQUAYlccqYbDgUGkTDISWEUFSCIA7pYxVAACpwAAaTcRWSYARYqHS2HeTE0HCgTAAsnS2AApKBGwX5usNrMkciUMjl2NC-P00IsFGEOgcVBH9UilgJXKFNhkMioFiIX2+mArgBWG4YHQLBkgExAYNEvrpBQoG+suzbrkaGD-iwAD8+40BwoS5P2A79CIi4AMr9Iui5ilmAASvbkYIZFiuyJ7EtUMYameYSXiw163veTEioUHA+EEHYJsQHYYAha4btUS4rkhGCUQoGQYAAigocDqX0ADkvbYKgcDQmQWkSJJ8mKRkTwANyMRqLGCnZJ7bhU5ZgBKoQcKBgRMAASnAoRSCwLAcOUADCxAtFhhAcIeojoDUrGNPmpqFNSdAYEwIV1qk0ILkUnSpKSZD9CU7ZyK6HDxVuHAYFOxAKGQWZBbIjp0MebHVQ4EDZI1HDNZ0bU2tuIwuSUIHlFUYAOXG+ZcHEDUmPAN6EEEvVIoUwKFBIPLZGtYAiOEm2zm26lIiMcwSDAXBIudTB-km+QVFcN2gXAqBIhUU2DdVs31VmC1lctq35AJEDRHAWZcFmADWh3bTkwNgAAkqDYJMCtRLNnVLCOPKzz4EdJZwEiGw3ZdRL5KTG5IlcJMSC9b0PZ91oze0v2JHAhBQxkxBGitjYI28c5Y1mEKJMksOZDtCPLrITD2nAQsKJYRTxGL0KHZox1E-kvZ0OpTOaturUG+1GBsMNdqlOUVyVXxwos3Nf2mADfO7cCNv43Du37bo+Oa4Tp2k1dFMXVTD1PXTFAM5UJsig7v3-UtrsI6C8AQxA0Me1tkvwwUSMo-AaNXZjSs4yweMSP7J35LTt3B0wlP3UwNPPVH72x-b32sw17Oc9zvNA3ngvBUrIuq1CcBZ2kOe7TLRPy4ryvj+Lfta0iuv6wl1oPg0HdMEljjdX+XADdvduntV5sjbi6jBRACqTVvhvVXVH6-fSGQFS5bwfBQHiPzvC+ZtjZP1NlfQoDJBAwAUOkB+e9iRJRcgAIWyGJVwUgmx1iYDQKBMCXBKxnPiJg4RhDNiJj-PEM43IyjlCwCQOo6GuCcHLB0ToXRumwYIJg9ZEhyytkSdE4UaCRWiuQSwDAFBOEIYiLqnRQJMBGGODmfA2y+GIFwMglgcF0DHB5ScPRqgYFttNDqLkAAkZidBWJ0BQI0ZAQDUIYZGJEQxRhjEGEwQMaM9grCYNiaxNjjT2OBJGGSgYgh7AOH4go1jbH2NTuDSGUNQmeLRucS4ij-GxKCSAYewtRYT1CRY4xX0sDmMsdkuxIAnFIjgAAPTZJU+x8tf5UMlDQ+UkZIzFIAefBBpjCjFKaSAe0xBHQZGdK6bAzjgbWPoGoEA9TGlWLiSMm+f8mCONoV0yMIAKhMH6NwhpATVktMoY2LZnSuk9PgY0QBBsHKEk8FInw0I6DaFaj9MgEhX5zTpgZIKpBaiCmkK0EkagNDaAwAANS1pSaQNI6SMmZHAIIEZBSCgAAK-PqliqQUi0ofwKiCloTQhForqOfUFSIqS0npEyWAqK2RgAAHKMofsqQoyMKAJFemQQ6hQfYCvzmDIuRIoYVzzgvEeS9IQr0tHGcU7SGEZSEVhQ8HKuT9A2KMQY2wtjnT2BsCoWwNiDHmIsK4j0jgSCuBgGmyxHobAVTabUtDVURUCKIzReoQymgGDq+4+rDWLGNaa81ewrVXBtc3e1GxHU0yeKEwBUgyR0CJNSrFOL+Unkxfi2QhLMjEpPN4HE7xzkUsAaC-Z+QKHBC+RgYEaLIjdwwAk9O0Nm28FbXk0eBTkhovTCKUFVwaXrPRnNRt4Qrhdobe2pJM6WLdsnb2lgY85XQkXdnXku1RplCJB7IdwpQUolJPCyCpBCB0j6NWvAI7+SuCNB5a66Lz6pvJGOGgGASFosAVm+qc08UyDkBgIlugS2kqgJWqlNAswUH4EiE9P6j1CibqCuDcB+BtoRIQFMeAKgPEATwfI6H4OYGdMQXDgICPPI1B2RDsGyPYco3hq4hHz4KAYxhrDFGqN4DY7RkUAAvLjTHeN4ZGOxjUABHUTmHyM4eAICSTf632knJOeulyLGWsiVVKBhHpsC+iNA+t1uokBcgUL6DspniisImew90FngDSd9EJh40ZAFxhFAmY0yZUyvuJNigDuLc35pA2BklPgMjQY1KR+TDHv1npQ8SNDjH5PMdw+dGj59iOfu4wplj1H2OAPoyR9LPHFPnQExxuTlWiv8ZK+fET5WCuZYk1JkUsnWticU8pzroo1NpopLSpFDKWTMsuZq4A2B+gmYkGZ6bCh+g2YW3ZsZbCpkemk-0dznm+neaFL5puKY0yqeeWgaOYhTSXZzMoVgcgPmEjAJGB4QA) answer the following questions: 

1. What are the predicted **log odds** for  `Person 1` who is a 30 year old, who makes $50k a year, and has made a previous purchase. What are the predicted **log odds** for `Person 2` to have all the **same** values, *except* they're 31. 
2. What are the predicted **log odds** for  `Person 1` who is a 72 year old, who makes $150k a year, and has made a previous purchase. What are the predicted **log odds** for `Person 2` to have all the **same** values, *except* they're 73. 



You can play around with other values, always making `Person 2` the same as `Person 1` except they're 1-year older.

- Is the ratio of their `Log Odds` ($\frac{\text{Person 2}}{\text{Person 1}}$) always the same? Is the Difference ($\text{Person 1} - \text{Person 2}$) of their `Log Odds` always the same?
- Is the ratio of their `Odds` ($\frac{\text{Person 2}}{\text{Person 1}}$) always the same? Is the Difference of their `Odds` ($\text{Person 1} - \text{Person 2}$) always the same?
- Is the ratio of their `Probabilities` ($\frac{\text{Person 2}}{\text{Person 1}}$) always the same? Is the Difference of their `Probabilities` ($\text{Person 1} - \text{Person 2}$) always the same?


Play around with different ages. Is either the *difference* or the *ratio* of the probabilities constant no matter what age you start with?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" style="width: 200px;"/>

Does the *ratio* of the predicted Odds or the *difference* for the predicted log odds look familiar? Do you see that number elsewhere?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" style="width: 200px;"/>

(Moral of the story probabilities can make more intuitive sense, but the amount a 1 unit increase in a predictor changes the predicted probability is *NOT CONSTANT*, it depends heavily on the values of the predictor of interest and all other predictors. If you're going to report in terms of probabilities you MUST be specific about what values all your predictors have.)